In [ ]:
## example

##cql
CREATE TABLE user_playlists (
    user_id UUID,
    song_added_date TIMESTAMP,
    song_id UUID,
    song_name TEXT,
    PRIMARY KEY (user_id, song_added_date)
);

#python
from cassandra.cluster import Cluster

cluster = Cluster(['localhost'])  # adjust ['localhost'] to your list of Cassandra nodes if different
session = cluster.connect('your_keyspace_name')  # replace with your keyspace name

# Define the materialized view query
mv_query = """
CREATE MATERIALIZED VIEW song_lookup AS
    SELECT user_id, song_added_date, song_id, song_name
    FROM user_playlists
    WHERE song_id IS NOT NULL AND user_id IS NOT NULL AND song_added_date IS NOT NULL
    PRIMARY KEY (song_id, user_id, song_added_date);
"""

# Execute the query
session.execute(mv_query)

##cql
SELECT * FROM song_lookup WHERE song_id = ?; #In this query, you'd replace the ? with the specific song_id value you're looking for.

## same query in python 

song_id_to_query = your_song_id_value  # replace this with the actual UUID value of song_id you want to query

# Query the materialized view
rows = session.execute('SELECT * FROM song_lookup WHERE song_id = %s', [song_id_to_query])

for row in rows:
    print(row.user_id, row.song_added_date, row.song_id, row.song_name)

cluster.shutdown()